![Podmanlogo](Pictures/podman-logo.png)

# From Podman to Kubernetes and viceversa

When working with kubernetes what most users want is a way of easily deploying in kubernetes what they have already tested in Podman. This can be done easily as Podman helps generating a yaml file that can be used afterwards in kubernetes. We demonstrated this on the Podman 101 workshop, but we are going to do it again and then we'll deploy it to an actual kubernetes cluster (in our case we chose OpenShift distribution).

Lets begin by creating a pod as we did before:

In [ ]:
podman pod create --name my-k8s-workload

We'll rung a Wordpress workload in the pod:

In [ ]:
podman run -d --pod my-k8s-workload \
  -e MYSQL_ROOT_PASSWORD="hpedev" \
  -e MYSQL_DATABASE="wp" \
  -e MYSQL_USER="wordpress" \
  -e MYSQL_PASSWORD="wordpress" \
  --name=wp-db \
  docker.io/library/mariadb:latest

podman run -d  --pod my-k8s-workload \
  -e WORDPRESS_DB_NAME="wp" \
  -e WORDPRESS_DB_USER="wordpress" \
  -e WORDPRESS_DB_PASSWORD="wordpress" \
  -e WORDPRESS_DB_HOST="127.0.0.1" \
  --name wp-web \
  docker.io/library/wordpress:latest

Check everything is working as expected:

In [ ]:
podman ps -a --pod

Now generate the kubernetes yaml file by using "podman generate kube" command:

In [ ]:
podman generate kube --replicas 2 --service --type deployment my-k8s-workload >> my-k8s-workload.yml

In previous command we have defined we want 2 replicas, we want the yaml file to include a service and to use a deployment object for deploying the workload. These concepts are not present in Podman but you can use them to specify the details of the file that Podman will generate and you'll use later in kubernetes. Take advantage of this flexibility!

Review the file we just generated:

In [ ]:
cat my-k8s-workload.yml

~~~
SECTION DEPLOYING THE WORKLOAD TO KUBERNETES, TO BE WRITTEN ONCE THE OPENSHIFT CLUSTER IS UP AND RUNNING
~~~

We've seen how you can use Podman to generate the files that you'll use later to deploy your workloads in kubernetes. But most probably you're thinking that kubernetes yaml files are super cool and you'd like to use them also for Podman workloads. No worries, Podman got your back.

Before moving on stop all of the running pods:

In [ ]:
podman pod rm --all -f

Now, if you wanted to run you pod in Podman using the kubernetes yaml file you just need to run the following command pointing to your yaml file:

In [ ]:
podman kube play my-k8s-workload.yml

You see it pops up a message explaining that it has reduced the amount of replicas to 1, that's because Podman is not dessigned to scale with multiple replicas. Check all of your containers have been created:

In [ ]:
podman ps -a --pod

That's cool, but there is even more. Remember the Quadlet files we used so systemd would work with our containers as if it was any other process? You can just pass the yaml file to the Quadlet so you don't need to manage different files with the definition of your workload.

You need to create a Quadlet file, but this time its name ends in ".kube" and it'll have the default "Install" section and a new one called "Kube". In this "Kube" section we will point to our yaml file and that's all we need to do for this example.

In [ ]:
cat << EOF > my-quadlet-yaml.kube
[Install]
WantedBy=default.target

[Kube]
# Point to the yaml file in the same directory
Yaml=my-k8s-workload.yml
EOF

Once you have created the file move it to "~/.config/containers/systemd/", you'll also need to move your yaml file as we used a relative path in the Quadlet definition.

In [ ]:
mv ~/my-k8s-workload.yml ~/.config/containers/systemd/my-k8s-workload.yml
mv ~/my-quadlet-yaml.kube ~/.config/containers/systemd/my-quadlet-yaml.kube

Now that all the files are present in the correct directory you just need to reload the systemctl daemon:

In [ ]:
systemctl --user daemon-reload

Finally, use systemctl to manage your containerized workload as a systemd unit:

In [ ]:
systemctl --user start my-quadlet-yaml.service

It's also possible to check the status:

In [ ]:
systemctl --user status my-quadlet-yaml.service --no-pager

Containers or pods managed by Quadlets will automatically start on system boot as well, so that's a cool way of using kubernetes yaml files also in your Podman environment.

# Clean up

In [ ]:
podman rm --all -f
podman pod rm --all -f
podman image prune -f
podman volume prune -f
podman network prune -f
rm -rf ~/.config/containers/systemd/*